# 16 Operator Overloading
Some notes, observations and questions along chapter 16.

- operator overloading allows user-defined objects to interoperate with infix operators such as `+` and `|`, or unary operators like `-` and `~`
- function invocation (`()`), attribute access (`.`), and item access/slicing (`[]`) are also operators that could be overloaded, but it's much more common with infix operators

- as a safety measure, 

    - Python doesn't allow us to overload `is`, `and`, `or` and `not`
    - we cannot create new operators, only overload the existing ones
    - we cannot change the meaning of operators for build-in types

### Unary Operators
- `-`, implemented by `__neg__`
- `+`, implemented by `__pos__`
- `~`, implemented by `__invert__`
    - bitwise not, or bitwise inverse of an integer, defined as `~x == -(x+1)`. If $x$ is $2$ then `~x == -3`
- `abs()`, implemented by `__abs__`

To code this, we take the corresponding special method that only takes `self` as an argument, and stick to the rule of always returning a new object: meaning we do not modify the receiver (self), but create and return a new instance of a suitable type.

In [ ]:
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self): # so __neg__ can iterate over self
        return iter(self._components)
    
    def __abs__(self):
        return math.hypot(*self)

    def __neg__(self):
        return Vector(-x for x in self)

    def __pos__(self): # we define the object as mutable, so we return a new object of the same type; 
        # we could just return the object, if it was immutable
        return Vector(self)

### Overloading + for Vector Addition
- we want to use `__add__` in a different way as usual: it should not concatenate two vectors, but instead add up the entries element-wise:

In [ ]:
def __add__(self, other):
        # `pairs` is a generator that produces tuples (a, b), where a is from self, and b is from other; if
        # self and other have different lengths, `fillvalue` fills the missing values for the shortest with 0s
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs) # __add__ returns a new Vector instance, and does not change self or other

In [ ]:
def __radd__(self, other):
        # added to facilitate mixed type additions, where our Vector() object is on the right side
        return self + other # just delegates to __add__

- however, this only works for commutative operations
- concatenation of two list like objects for instance, would not be commutative

In [ ]:
# another way to archive the same:

def __add__(self, other):
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a + b for a, b in pairs)

    __radd__ = __add__

- we would get errors

    - if we tried to add a non-iterative object: `TypeError: zip_longest argument #2 must support iteration`

    - or if we add an object, that doesn't implement `__add__`: `TypeError: unsupported operand type(s) for +: 'float' and 'str'`

- because these `TypeErrors` are not very clear (and because they depend on the implementation of the other object), we should better define:

In [ ]:
    def __add__(self, other):
        try:
            pairs = itertools.zip_longest(self, other, fillvalue=0.0)
            return Vector(a + b for a, b in pairs)
        except TypeError:
            return NotImplemented # to return a special type (not an error message), that allows the 
                                  # interpreter to try again by swapping the operands and calling the reverse 
                                  # special method for that operator
        
    def __radd__(self, other):
        return self + other

So, summing up what I understand between the lines:

Operator overloading means over-riding the functionality of a method that's inherited from a base class. The operator will then perform the operation we've defined instead of any default behavior.

- It does not meant there are two functionalities at play at the same time!

But then,what's the difference between method over riding and operator overloading?

- It seems operator overloading is just a special case of method over riding.

### Overloading * for Scalar Multiplication


In [15]:
    # inside the Vector class

    def __mul__(self, scalar):
        return Vector(n * scalar for n in self)

    def __rmul__(self, scalar):
        return self * scalar

In [16]:
    # in order to take care of multiplying with unsupported types we better add a way to catch the TypeErrors
    
    def __mul__(self, scalar):
            try:
                factor = float(scalar)
            except TypeError:
                return NotImplemented
            return Vector(n * factor for n in self)

    def __rmul__(self, scalar):
        return self * scalar

### Using @ as an Infix Operator

In [ ]:
    def __matmul__(self, other):
        if (isinstance(other, abc.Sized) and isinstance(other, abc.Iterable)): # goose typing with `isinstance()`
            if len(self) == len(other):
                return sum(a * b for a, b in zip(self, other)) # application of sum, zip, and generator expression
            else:
                raise ValueError('@ requires vectors of equal length.')
        else:
            return NotImplemented

    def __rmatmul__(self, other):
        return self @ other

### Rich Comparison Operators
- these are `==`, `!=`, `>`, `<`,` >=`, and `<=`
- their handling by the interpreter is similar to above but differs in two aspects
    - 1. same set of methods is used in forward and reverse operator calls
        - `==` invokes `__eq__` and the reverse is `__eq__` with the arguments swapped
        - `>=` invokes `__gt__` and the reverse is `__lt__` with the arguments swapped
    - 2. if in `==` or `!=` the reverse argument is missing, Python interpreter compares the object id's

### Augmented Assignment Operators
- inplace assingment; for instance  `+=` and `*=`
- for immutable classes `+=` works by calling `__add__` and the object is a new object (different id):

In [21]:
v1 = "some string"
v1_alias = v1
id(v1)

128697690460272

In [22]:
v1 += " and another string"
v1

'some string and another string'

In [23]:
id(v1)

128697688519936

- however, if we implement `__iadd__` we are expected to change the object (which makes it mutable)

    - if we really want that, `__iadd__` needs to be implemented by returning (an updated) self